# 9 - Predições das matrículas

Para as matrículas, deseja-se atingir uma meta em relação à população. É possível prever como o número de matrículas vai crescer, mas a meta em relação à população precisa da estimativa da população para acontecer. 

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')
dados_educ_pop = pd.read_csv('dados/dados_educ_pop.csv', sep = ',', parse_dates=['ANO_MES'], index_col='ANO_MES', date_parser=dateparse) 

# Só precisamos do distrito e da matrícula, para esse caso
dados_matricula = dados_educ_pop[['DISTRITO', 'MAT_CRECHE', 'DISTRITO_F']]

distritos = pd.read_csv('dados/distritos.csv', sep = ',')

Usando o modelo de predição ARIMA, sigla para Média móvel integrada autoregressiva, que forneceu o menor erro médio nos testes, vamos prever como as matrículas vão se comportar. Além disso, vamos ver quando a meta de atendimento pode ser batida, baseando-se na média de população entre 1995 e 2018.

In [16]:
# Pegando os dados populacionais de 1995 a 2018
dados_pop = pd.read_csv('dados/populacao2.csv', sep = ';', encoding = 'ISO-8859-1')
dados_pop = dados_pop[["Localidades", "População de 0 a 3 Anos"]]
dados_pop.columns = ['DISTRITO', 'TOTAL']

# Calculando a média
dados_pop = dados_pop.groupby(['DISTRITO'])['TOTAL'].mean().to_frame().reset_index()

Agora, vamos gerar os modelos preditivos e comparar os resultados com a média populacional.

In [17]:
import statsmodels.api as sm

lista = []

distritos['PERIODO'] = 'default value'

# Definindo a meta
meta = 0.75

for distrito in dados_matricula['DISTRITO'].unique():
    dados_matricula = dados_educ_pop[dados_educ_pop['DISTRITO'] == distrito]
    distrito_f = dados_matricula.at[dados_matricula.index[0], 'DISTRITO_F']

    # Gerando o modelo
    fit1 = sm.tsa.statespace.SARIMAX(dados_matricula['MAT_CRECHE'], order=(2, 1, 4),seasonal_order=(0,1,1,4)).fit()
    matricula_prevista = fit1.predict(start="2018-03-01", end="2030-12-01", dynamic=True)

    # Pegando a média populacional
    pop = dados_pop[dados_pop['DISTRITO'] == distrito_f]['TOTAL'].values[0]

    # Verificando se a meta foi atingida
    atingiu = False
    for index, element in matricula_prevista.iteritems():
        if element/pop >= meta:
            data = str(index)
            periodo = data[:4] + '-' + data[5:7]
            atingiu = True
            break
    if not atingiu:
            periodo = '2031-03'
            
    i = distritos[distritos['DISTRITO_F'] == distrito_f].index[0]    
    distritos.at[i, 'PERIODO'] = periodo

#     # Gerando o gráfico
#     plt.ioff()
#     fig = plt.figure(dpi=200, figsize=(10, 6))
#     plt.title('Evolução das matrículas das creches em ' + distrito)
#     plt.plot(dados_matricula['MAT_CRECHE'], label='Matrículas registradas nas creches do distrito', color='c', linewidth=5.0)
#     plt.plot(matricula_prevista, label='Matrículas previstas nas creches do distrito', color='midnightblue', linewidth=5.0)
#     plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),  shadow=True, ncol=2)
#     plt.ylabel('Crianças de 0 a 3 anos')
#     plt.xlabel('Data de Referência')
#     plt.xticks(rotation=90)
#     plt.savefig('graficos/previsao_matricula_' + distrito + '.png', bbox_inches='tight')
#     plt.close(fig)

/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/statespace/sarimax.py:960: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was 

/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequ

/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_r

/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequ

/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels-0.10.0.dev0+868.g14fb572ce-py3.6-linux-x86_64.egg/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequ

In [18]:
distritos = distritos.sort_values(by='PERIODO').reset_index(drop=True)
for index in distritos.index:
    posicao = str(index+1)
    zona = distritos.at[index, 'ZONA']
    nome = distritos.at[index, 'DISTRITO_F']
    periodo = distritos.at[index, 'PERIODO']
    print("\\textbf{" + posicao + "} & " + nome + " & " + zona + " & " + periodo + "\\\\ \hline")

\textbf{1} & Lajeado & Leste & 2018-03\\ \hline
\textbf{2} & Jaguara & Oeste & 2018-03\\ \hline
\textbf{3} & Guaianases & Leste & 2018-03\\ \hline
\textbf{4} & Cidade Tiradentes & Leste & 2018-03\\ \hline
\textbf{5} & Marsilac & Sul & 2018-06\\ \hline
\textbf{6} & Jaraguá & Norte & 2018-09\\ \hline
\textbf{7} & Anhanguera & Norte & 2018-12\\ \hline
\textbf{8} & Itaim Paulista & Leste & 2020-06\\ \hline
\textbf{9} & Perus & Norte & 2020-09\\ \hline
\textbf{10} & Iguatemi & Leste & 2020-09\\ \hline
\textbf{11} & Parelheiros & Sul & 2020-12\\ \hline
\textbf{12} & Brasilândia & Norte & 2021-06\\ \hline
\textbf{13} & Vila Sônia & Oeste & 2023-03\\ \hline
\textbf{14} & Jardim Helena & Leste & 2023-03\\ \hline
\textbf{15} & Jaguaré & Oeste & 2023-03\\ \hline
\textbf{16} & Cidade Dutra & Sul & 2023-03\\ \hline
\textbf{17} & Sacomã & Sul & 2023-09\\ \hline
\textbf{18} & Cangaiba & Leste & 2024-03\\ \hline
\textbf{19} & Raposo Tavares & Oeste & 2024-03\\ \hline
\textbf{20} & Jardim São Luís & Su